In [95]:
import warnings

# 오류 경고 무시하기
warnings.filterwarnings(action='ignore')


In [96]:
import pandas as pd
df = pd.read_csv('한식음식점.csv')
# df.head()

In [97]:
def create_geodata(df):
    import pandas as pd
    import geopandas as gpd
    import matplotlib.pyplot as plt
    
    # prepare and read data
    F1 = 'data/상권 영역/서울시 상권분석서비스(영역-상권).shp' 
    D1 = gpd.read_file(F1, encoding='utf-8')

    geo_data = pd.merge(df[['상권_코드_명','총_유동인구_수','총_상주인구_수','총_직장_인구_수','유사_업종_점포_수','집객시설수','교통시설수','당월_매출_금액','영역_면적','cluster']],D1,left_on='상권_코드_명',right_on='TRDAR_CD_N')
    geo_data = geo_data[['상권_코드_명','총_유동인구_수','총_상주인구_수','총_직장_인구_수','유사_업종_점포_수','집객시설수','교통시설수','당월_매출_금액','영역_면적','cluster','geometry']]
    geo_data = gpd.GeoDataFrame(geo_data)

    return geo_data

In [98]:
geo_data = pd.merge(df[['상권_코드_명','총_유동인구_수','총_상주인구_수','총_직장_인구_수','유사_업종_점포_수','집객시설수','교통시설수','당월_매출_금액','영역_면적','cluster']],D1,left_on='상권_코드_명',right_on='TRDAR_CD_N')
geo_data = geo_data[['상권_코드_명','총_유동인구_수','총_상주인구_수','총_직장_인구_수','유사_업종_점포_수','집객시설수','교통시설수','당월_매출_금액','영역_면적','cluster','geometry']]
geo_data = gpd.GeoDataFrame(geo_data)
# geo_data

In [99]:
import geopandas as gpd

# Assuming 'your_column' is the column you're working with
if isinstance(geo_data['geometry'], gpd.geoseries.GeoSeries):
    geo_data['geometry'] = geo_data['geometry'].buffer(5)
    geo_data['result'] = None
    
    cluster_data = {}  # Dictionary to store DataFrames

    for i in range(geo_data['cluster'].nunique()):
        cluster_data[f'geo_data_{i}'] = geo_data[geo_data['cluster'] == i].reset_index(drop=True)

else:
    print("It's not a GeoSeries.")

In [100]:
def merge_polygon(geo_data):
    import geopandas as gpd
    from shapely.ops import unary_union
    data = dict(상권_코드=[], 총_유동인구_수=[], 총_상주인구_수=[], 총_직장_인구_수=[], 유사_업종_점포_수=[], 집객시설수=[], 교통시설수=[], 당월_매출_금액=[],영역_면적=[], cluster=[],geometry=[])
    df = gpd.GeoDataFrame(data)
    n=1 
    cluster = geo_data['cluster'][0]
    # geo_data['result']=None
    while len(geo_data) != 0:
        while geo_data['result'].sum() != 1:
            geo_data['result'] = None
            for row in range(len(geo_data)):
                if geo_data['geometry'][0].intersects(geo_data['geometry'][row]):
                    geo_data['result'][row]=1
                else :
                    geo_data['result'][row]=0
            geo_data_union = geo_data[geo_data['result']==1]
            mergedPolys = unary_union(geo_data_union['geometry'])
            geo_data = geo_data[geo_data['result']==0]
            geo_data.loc[0] = [n,geo_data_union['총_유동인구_수'].sum(),geo_data_union['총_상주인구_수'].sum(),geo_data_union['총_직장_인구_수'].sum(),geo_data_union['유사_업종_점포_수'].sum(),geo_data_union['집객시설수'].sum(),geo_data_union['교통시설수'].sum(),geo_data_union['당월_매출_금액'].sum(),geo_data_union['영역_면적'].sum(),cluster,mergedPolys,1]
        df.loc[len(df)] = [n,geo_data['총_유동인구_수'][0],geo_data['총_상주인구_수'][0],geo_data['총_직장_인구_수'][0],geo_data['유사_업종_점포_수'][0],geo_data['집객시설수'][0],geo_data['교통시설수'][0],geo_data['당월_매출_금액'][0],geo_data['영역_면적'][0],cluster,geo_data['geometry'][0]]
        geo_data = geo_data.drop(0, axis=0).reset_index(drop=True)
        n+=1
    return df

In [101]:
merge_polygon_data = {}  # Dictionary to store DataFrames

for i in range(len(cluster_data)):
    merge_polygon_data[f'df_{i}'] = merge_polygon(cluster_data[f'geo_data_{i}'])


In [102]:
def concat_geodata(merge_polygon_data):
    for i in range(1,len(merge_polygon_data)):
        if i == 1:
            df = pd.concat([merge_polygon_data['df_0'],merge_polygon_data['df_1']],axis=0)
        else:
            df = pd.concat([df,merge_polygon_data[f'df_{i}']],axis=0)
    list = []
    for i in range(1,len(df)+1):
        list.append(i)
    df['상권_코드'] = list
    df = df.reset_index(drop=True)
        
    return df
    
# df_korean_food = pd.concat([merge_polygon_data['df_0'],merge_polygon_data['df_1'],merge_polygon_data['df_2']],axis=0)
# df_korean_food = pd.concat([df_korean_food,merge_polygon_data['df_2']],axis=0)
# df_korean_food = pd.concat([df_korean_food,df_3],axis=0)
# df_korean_food = df_korean_food.reset_index(drop=True)
# len(df_korean_food)
# df_korean_food

In [103]:
df_all = concat_geodata(merge_polygon_data)

#### 평가 점수 부여

In [104]:
def evaluation_score(df,score1,score2,score3,score4,score5_1,score5_2,score5_3):
    df[['집객력(점수)','매출액(점수)','경쟁업체(점수)','편의성(점수)','잠재고객(점수)']] = 0
    # 집객력
    for i in range(len(df)):
        if df['집객시설수'][i].sum()<=score1[0]:
            df['집객력(점수)'][i] = 1
        elif df['집객시설수'][i].sum()<=score1[1]:
            df['집객력(점수)'][i] = 2
        elif df['집객시설수'][i].sum()<=score1[2]:
            df['집객력(점수)'][i] = 3
        elif df['집객시설수'][i].sum()<=score1[3]:
            df['집객력(점수)'][i] = 4
        else:
            df['집객력(점수)'][i] = 5
        
        # 매출액
        if df['당월_매출_금액'][i].sum()<=score2[0]:
            df['매출액(점수)'][i] = 1
        elif df['당월_매출_금액'][i].sum()<=score2[1]:
            df['매출액(점수)'][i] = 2
        elif df['당월_매출_금액'][i].sum()<=score2[2]:
            df['매출액(점수)'][i] = 3
        elif df['당월_매출_금액'][i].sum()<=score2[3]:
            df['매출액(점수)'][i] = 4
        else:
            df['매출액(점수)'][i] = 5
    
        # 경쟁업체
        if df['유사_업종_점포_수'][i].sum()<=score3[0]:
            df['경쟁업체(점수)'][i] = 1
        elif df['유사_업종_점포_수'][i].sum()<=score3[1]:
            df['경쟁업체(점수)'][i] = 2
        elif df['유사_업종_점포_수'][i].sum()<=score3[2]:
            df['경쟁업체(점수)'][i] = 3
        elif df['유사_업종_점포_수'][i].sum()<=score3[3]:
            df['경쟁업체(점수)'][i] = 4
        else:
            df['경쟁업체(점수)'][i] = 5
    
        # 편의성
        if df['교통시설수'][i].sum()<=score4[0]:
            df['편의성(점수)'][i] = 1
        elif df['교통시설수'][i].sum()<=score4[1]:
            df['편의성(점수)'][i] = 2
        elif df['교통시설수'][i].sum()<=score4[2]:
            df['편의성(점수)'][i] = 3
        elif df['교통시설수'][i].sum()<=score4[3]:
            df['편의성(점수)'][i] = 4
        else:
            df['편의성(점수)'][i] = 5
    
        # 잠재고객
        ## 상주인구
        if df['총_상주인구_수'][i].sum()<=score5_1[0]:
            df['잠재고객(점수)'][i] += 1
        elif df['총_상주인구_수'][i].sum()<=score5_1[1]:
            df['잠재고객(점수)'][i] += 2
        elif df['총_상주인구_수'][i].sum()<=score5_1[2]:
            df['잠재고객(점수)'][i] += 3
        elif df['총_상주인구_수'][i].sum()<=score5_1[3]:
            df['잠재고객(점수)'][i] += 4
        else:
            df['잠재고객(점수)'][i] += 5
        
        ## 유동인구
        if df['총_유동인구_수'][i].sum()<=score5_2[0]:
            df['잠재고객(점수)'][i] += 1
        elif df['총_유동인구_수'][i].sum()<=score5_2[1]:
            df['잠재고객(점수)'][i] += 2
        elif df['총_유동인구_수'][i].sum()<=score5_2[2]:
            df['잠재고객(점수)'][i] += 3
        elif df['총_유동인구_수'][i].sum()<=score5_2[3]:
            df['잠재고객(점수)'][i] += 4
        else:
            df['잠재고객(점수)'][i] += 5
        
        ## 직장인구
        if df['총_직장_인구_수'][i].sum()<=score5_3[0]:
            df['잠재고객(점수)'][i] += 1
        elif df['총_직장_인구_수'][i].sum()<=score5_3[1]:
            df['잠재고객(점수)'][i] += 2
        elif df['총_직장_인구_수'][i].sum()<=score5_3[2]:
            df['잠재고객(점수)'][i] += 3
        elif df['총_직장_인구_수'][i].sum()<=score5_3[3]:
            df['잠재고객(점수)'][i] += 4
        else:
            df['잠재고객(점수)'][i] += 5

    df['잠재고객(점수)'] = df['잠재고객(점수)'] / 3
    
    return df
        
        

In [105]:
score1 = [3,5,12,23]
score2 = [270000000,630000000,1280000000,2900000000]
score3 = [8,16,26,49]
score4 = [0,2,5,9]
score5_1 = [500,668,1655,3361]
score5_2 = [100000,230000,580000,1000000]
score5_3 = [50,111,321,894]

In [106]:
df_final = evaluation_score(df_all,score1,score2,score3,score4,score5_1,score5_2,score5_3)

In [107]:
df_final.head()

,상권_코드,총_유동인구_수,총_상주인구_수,총_직장_인구_수,유사_업종_점포_수,집객시설수,교통시설수,당월_매출_금액,영역_면적,cluster,geometry,집객력(점수),매출액(점수),경쟁업체(점수),편의성(점수),잠재고객(점수)
0,1,5724917.75,16670,1487,163,0.0,0.0,8.969312e+09,384314,0,"POLYGON ((191627.514 450613.571, 191630.569 45...",1,5,5,1,5.000000
1,2,4547479.75,14552,2169,173,143.0,19.0,1.094268e+10,497570,0,"POLYGON ((211319.234 448645.070, 211324.441 44...",5,5,5,5,5.000000
2,3,6274558.25,7311,3873,298,160.0,30.0,2.258324e+10,361277,0,"POLYGON ((201732.035 459543.556, 201732.443 45...",5,5,5,5,5.000000
3,4,1598847.25,5758,537,54,0.0,0.0,1.578294e+09,191612,0,"POLYGON ((186573.498 444690.705, 186559.573 44...",1,4,5,1,4.666667
4,5,2671650.50,7559,2069,85,45.0,6.0,5.948167e+09,241319,0,"POLYGON ((212860.894 443951.859, 212861.265 44...",5,5,5,4,5.000000
